# Reverse Engineering the UR3e IK parameters

Universal Robots posts the [DH parameters](https://www.universal-robots.com/articles/ur/application-installation/dh-parameters-for-calculations-of-kinematics-and-dynamics/) of all of their robots online, the hope is with these parameters for our robot, and the joint position read off the modbus, we might be able to reverse-engineering the inverse kinematics and write our own controller for working with a floating coordinate system.

In [34]:
import numpy as np
import json
import os
import time

np.set_printoptions(formatter={'float': '{: 0.3f}'.format})


In [35]:
DH = np.array([[0, 0, 0.1519,np.pi/2],
                    [0,-0.2434,0.,0.],
                    [0,-0.2132,0.,0.],
                    [0,0.,0.1311,np.pi/2],
                    [0,0.,0.0854,-np.pi/2],
                    [0,0.,0.0921,0.]])

print("Available files:")
files = []
for file in os.listdir():
    if file.endswith(".json"):
        # print("\t" + file)
        files.append(file)

i = 8

path = os.path.abspath("..\\IK_Scans") + "\\" + files[i]

print(path)

with open(path, 'r') as infile:
    json_data = json.load(infile)


start_joints = json_data["Start joints"]
joints_at_points = json_data["joints_at_points"]

print(np.rad2deg(start_joints))
names = ("Floating coordinate system", "Joint angles", "Base position")
for i in range(3):
    print(f"{names[i]}: {joints_at_points[0][i]}")
    
# print(joints_at_points[1])

Available files:
c:\Users\ander\Documents\COBOT-Transducer-Control-Code\IK_Scans\Test_08_11__15_06.json
[[ 166.673]
 [ 307.736]
 [ 94.882]
 [ 228.266]
 [ 273.645]
 [ 107.315]]
Floating coordinate system: [[0, 0, 0], [0, 0, 0]]
Joint angles: [[2.909], [5.371], [1.656], [3.984], [4.776], [1.873]]
Base position: [[[0.4106], [0.0533], [-0.0662]], [[-0.822], [-3.011], [-0.089]]]


In [40]:
#               theta,a,d,alpha     (radians and meters)
DH = np.array([[0, 0, 0.1519,np.pi/2],
                    [0,-0.2434,0.,0.],
                    [0,-0.2132,0.,0.],
                    [0,0.,0.1311,np.pi/2],
                    [0,0.,0.0854,-np.pi/2],
                    [0,0.,0.0921,0.]])

def matrix_to_coordinates(mat):
    # print("debugging the dumb method :/")
    X = mat[0,3]
    Y = mat[1,3]
    Z = mat[2,3]

    # print(mat)
    # print(f"mat[2,0]: {mat[2,0]}")

    beta = np.arctan2((mat[0,0]**2 + mat[1,0]**2)**0.5,-mat[2,0])
    cosb=np.cos(beta)
    alpha = np.arctan2(mat[0,0]/cosb, mat[1,0]/cosb)
    gamma = np.arctan2(mat[2,2]/cosb, mat[2,1]/cosb)
    return X,Y,Z,beta,alpha,gamma

def joints_forward_kinematics(joints):
    '''Accepts as input a list of 6 joint positions in radians'''
    joints = np.array(joints).flatten()
    DH_params = DH.copy()
    DH_params[:,0] = start_joints

    j_pos = []
    for i in range(6):
        cos_io = np.cos(DH_params[i,0])
        sin_io = np.sin(DH_params[i,0])
        j_pos.append(np.array([[cos_io, -sin_io * np.cos(DH_params[i,3]), sin_io * np.sin(DH_params[i,3]), DH_params[i,1] * cos_io],
                                [sin_io, cos_io * np.cos(DH_params[i,3]), -cos_io * np.sin(DH_params[i,3]), DH_params[i,1] * sin_io],
                                [0., np.sin(DH_params[i,3]), np.cos(DH_params[i,3]),DH_params[i,2]],
                                [0.,0.,0.,1.]]))
    
    (T1,T2,T3,T4,T5,T6) = tuple(j_pos)
    t = T1
    for i in range(5):
        t = np.matmul(t,j_pos[i+1])

    return matrix_to_coordinates(t)

start_joints = np.array(start_joints).flatten()
DH[:,0] = start_joints
# print(DH)

joints=[]


for i in range(6):
    joints.append(np.array([[np.cos(DH[i,0]), -np.sin(DH[i,0]) * np.cos(DH[i,3]), np.sin(DH[i,0]) * np.sin(DH[i,3]), DH[i,1] * np.cos(DH[i,0])],
                            [np.sin(DH[i,0]), np.cos(DH[i,0]) * np.cos(DH[i,3]), -np.cos(DH[i,0]) * np.sin(DH[i,3]), DH[i,1] * np.sin(DH[i,0])],
                            [0., np.sin(DH[i,3]), np.cos(DH[i,3]),DH[i,2]],
                            [0.,0.,0.,1.]]))

(T1,T2,T3,T4,T5,T6) = tuple(joints)
# print(T1)
t = T1
for i in range(5):
    print(i)
    # print(f"Current matrix: {t}")
    # print(f"Current coordinates: {matrix_to_coordinates(t)}")
    t = np.matmul(t,joints[i+1])

# final_result = np.matmul(np.matmul(np.matmul(np.matmul(np.matmul(T1,T2),T3),T4),T5),T6)

print("-----------------")
print(t)

print(matrix_to_coordinates(t))
# [format(a,'1.2f') for a in joints_forward_kinematics(start_joints)]
print([format(a,'0.5f') for a in joints_forward_kinematics(start_joints)])

0
1
2
3
4
-----------------
[[-0.860  0.510 -0.000  0.411]
 [ 0.509  0.858  0.065  0.043]
 [ 0.034  0.056 -0.998  0.107]
 [ 0.000  0.000  0.000  1.000]]
(0.4109003305853896, 0.04341059041693143, 0.10681338241267886, 1.6044459540027232, 2.105138391218879, 1.6269468950194426)
['0.41090', '0.04341', '0.10681', '1.60445', '2.10514', '1.62695']
